In [1]:
from graph_ds import PyH3Graph
import h3.api.numpy_int as h3

In [2]:
graph = PyH3Graph(weight_options={}, k_ring=2, layers="all")
graph.create(osm_path="../resources/copenhagen/copenhagen-processed.osm.pbf", gtfs_paths=["../resources/copenhagen/rejseplanen.zip"])

processing osm pbf file: ../resources/copenhagen/copenhagen-processed.osm.pbf
converted OSM file into 490098 edges
osm graph created with 418149 nodes in 2.2872908 s
getting GTFS feed from ../resources/copenhagen/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 10.261178 s
merged gtfs graph into osm graph, now has 519554 nodes, took 180 ms
hash: 8759942521107932551


In [5]:
graph = PyH3Graph(weight_options=dict({
    "bike_penalty": 1.0,
    "wait_time_multiplier": 1.0,
    "walk_speed": 1.4,
    "bike_speed": 4.5
}), k_ring=2, layers="all")
graph.create(osm_path="../resources/copenhagen/copenhagen-processed.osm.pbf", gtfs_paths=["../resources/copenhagen/rejseplanen.zip"])

processing osm pbf file: ../resources/copenhagen/copenhagen-processed.osm.pbf
converted OSM file into 490098 edges
osm graph created with 418149 nodes in 2.0844584 s
getting GTFS feed from ../resources/copenhagen/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 9.477737 s
merged gtfs graph into osm graph, now has 519554 nodes, took 157 ms
hash: 8759942521107932551


In [6]:
graph.matrix_distance(
    origins=[631049690183383551, 631049690224672255, 631049690224671743, 631049690149913599], 
    destinations=[631051737789436415, 631049684202690047, 631049684202697215], 
    hour_of_week=None,
    infinity=45.0,
    dynamic_infinity=True
)

{631049690224672255: [None, None, None],
 631049690224671743: [None, None, None],
 631049690149913599: [None, None, None]}